In [ ]:
#Run this cell to install the necessary packages
import subprocess
import pkg_resources

def install_if_needed(package, version):
    '''Function to ensure that the libraries used are consistent to avoid errors.'''
    try:
        pkg = pkg_resources.get_distribution(package)
        if pkg.version != version:
            raise pkg_resources.VersionConflict(pkg, version)
    except (pkg_resources.DistributionNotFound, pkg_resources.VersionConflict):
        subprocess.check_call(["pip", "install", f"{package}=={version}"])

install_if_needed("langchain-core", "0.3.18")
install_if_needed("langchain-openai", "0.2.8")
install_if_needed("langchain-community", "0.3.7")
install_if_needed("unstructured", "0.14.4")
install_if_needed("langchain-chroma", "0.1.4")
install_if_needed("langchain-text-splitters", "0.3.2")

In [19]:
from dotenv import load_dotenv
import os
load_dotenv()
openai_api_key = os.environ["OPENAI_API_KEY"]

# Import the required packages
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [15]:
loader = UnstructuredHTMLLoader(file_path="data/mg-zs-warning-messages.html")
car_docs = loader.load()


In [22]:
# Initialize RecursiveCharacterTextSplitter to make chunks of HTML text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(car_docs)

for i, split in enumerate(splits):
    print(f"--- Chunk {i+1} ---")
    print(split)
    print()

--- Chunk 1 ---
page_content='Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer as soon as possible. Engine Coolant Temperature High High engine coolant temperature could result in severe damage. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Engine Coolant Temperature Sensor Fault Indicates that the engine coolant temperature sensor has failed. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately.' metadata={'source': 'data/mg-zs-warning-messages.html'}

--- Chunk 2 ---
page_content='Warning Message Procedure Low Oil Pressure Indicates that the oil pressure is too low, which may re

In [27]:
# Initialize Chroma vectorstore with documents as splits and using OpenAIEmbeddings
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=openai_api_key),persist_directory="vectorstore_db")

In [28]:
# Setup vectorstore as retriever
retriever = vectorstore.as_retriever()

In [29]:
results = retriever.get_relevant_documents("what is the warning message for the MG-ZS car?")
for result in results:
    print(result.page_content)

Warning Message Procedure Airbag Fault Indicates that the Airbag system has detected a fault. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Front Left/Front Right/Rear Left/Rear Right Tyre Sensor Battery Low Indicates that the Tyre Pressure Monitoring system has detected that a tyre pressure sensor has a low battery. Please consult an MG Authorised Repairer as soon as possible. TPMS Fault Indicates that the tyre pressure monitoring system has detected a fault. Please consult an MG Authorised Repairer as soon as possible.

Warning Message Procedure 12V Battery Charging System Fault Indicates that the 12V battery charging system has failed. Please consult an MG Authorised Repairer immediately. Rear Drive Assist System Fault Indicates that the rear drive assist system (RDA) has detected a fault. Please consult an MG Authorised Repairer as soon as possible.
Warning Message Procedure Stop Start System Fault

In [30]:
# Define RAG prompt
prompt = ChatPromptTemplate.from_template("You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:")

In [31]:
model = ChatOpenAI(openai_api_key=openai_api_key, model_name="gpt-4o-mini", temperature=0)


In [32]:
# Setup the chain
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | model
)

In [33]:
# Initialize query
query = "The Gasoline Particular Filter Full warning has appeared. What does this mean and what should I do about it?"

In [34]:
# Invoke the query
answer = rag_chain.invoke(query).content
print(answer)

The Gasoline Particular Filter Full warning indicates that the gasoline particulate filter is full. You should consult an MG Authorised Repairer as soon as possible for assistance.
